In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import scipy.sparse as sp

# Basics of Continuation

## Ressources
- Lecture notes Hermann Riecke [pdf](https://www.researchgate.net/publication/228688534_Methods_of_Nonlinear_Analysis_412), [videos](https://youtu.be/VDtvmMSZZjI?si=th6DVQIRiyyPA9UP)
- Seydel. Practical bifurcation and stability analysis. (2009)
- Uecker. Continuation and Bifurcation in Nonlinear PDEs – Algorithms, Applications, and Experiments (2022)
- Kurt Lust PhD thesis on continuation of periodic orbits

## Main idea

Consider a dynamical system described by the following (system of coupled) ODE(s),

$$
\dfrac{d}{dt} \bm{u} = \bm{F}(\bm{u}, \eta),
$$

where $\bm{u}$ is the state vector and $\eta$ the control parameter. In the case of a PDE or integro-differential equation, it can be discretized in space and approximated by a system of ODEs,
corresponding to the above equation.

The main objective of numerical continuation, is to obtain both stable and unstable equilibria of the dynamical system as the parameters are varied (bifurcation diagram). In what follows, we will restrict the analysis to only steady states or fixed point, for simplicity. Nevertheless, the method can be extended to find limit cycles as well.


### Finding the equilibria

Steady states do not evolve in time, therefore they satisfy the following (system of nonlinear) equation(s),

$$
\bm{F}(\bm{u}, \eta) = 0.
$$

The above equation can be solved through various methods, the preferred one is Newton-Raphson's for its fast (quadratic) convergence and simplicity. Consider an initial guess $\bm{u}_0$ close enough to the 'true' solution, succesive points will be determined through the following Eqs.

$$
\bm{J}(\bm{u}_i, \eta) \Delta \bm{u}_{i+1} = -\bm{F}(\bm{u}_i, \eta) \\
\bm{u}_{i+1} = \bm{u}_i + \Delta \bm{u}_{i+1}
$$

Where $\bm{J}$ is the Jacobian of $\bm{F}$. Since the Jacobian is computed at each iteration, it can be used to find the stability and detect bifurcation points!

We now know how to find a solution for a fixed parameter value, let's see how it can be changed.


### Varying the parameter

The whole point of the continuation is to obtain the bifurcation diagram, i.e. the solution as the parameter is varied. There are several strategies, the simplest and most frequent are the following.

#### Natural parameter continuation

The most basic strategy would be to find the solution $\bm{u}_0$ for $\eta = \eta_0$, then take a small step $\Delta \eta$, and find the solution $\bm{u}_1$ for $\eta = \eta_0 + \Delta \eta$
using the previous solution as initial guess. 

Let's illustrate the method with a simple example, consider the imperfect pitchfork bifurcation with $\varepsilon = 1$.

$$
\dfrac{du}{dt} = \eta + \varepsilon u - u^3
$$

Find the solutions $u(\eta)$ for $\eta \in [-1, 1]$


It turns out that the control parameter is not necessarily the best way to parametrize the solution curve.

#### Pseudo-arclength continuation

The PALC algorithm aims to parametrize the solution curve by the arclength $s$, through a predictor-corrector method.

Consider we previously know a point of the solution branch $\bm{y}_0 = (\bm{u}_0, \eta_0)$ and the tangent at that point $\bm{\tau}_0$. 

1. *Predictor step*: Extrapolate a distance $\Delta s$ along the tangent $\bm{\tau}_0$ from the previously known point  $(\bm{u}_0, \eta_0)$ an use it as initial guess

$$
\bm{y} = \bm{y}_0 + \bm{\tau}_0 \Delta s
$$

2. *Corrector step*: Force the solution to stay at a distance $\Delta s$ in the plane perpendicular to the tangent.
$$
(\bm{y} - \bm{y}_0) \cdot \bm{\tau}_0 = \Delta s
$$

What is the tangent vector?

$$
\bm{\tau} = \dfrac{d}{ds} \bm{y} = \left(\dfrac{d\bm{u}}{ds}, \dfrac{d\eta}{ds}\right)^T
$$

How can we compute it? Recall that we want to solve the following equation. It is useful to write out the dependance on $s$,

$$
\bm{F}(\bm{u}(s), \eta(s)) = 0.
$$

Then take the derivative with respect to $s$,

$$
\bm{J}(\bm{u}(s), \eta(s)) \dfrac{d\bm{u}}{ds} + \bm{F}_\eta(\bm{u}(s), \eta(s)) \dfrac{d\eta}{ds} = 0.
$$

In order to find a unique solution to the above equation, we need to add another restriction: that the tangent is normalized. For this, it is convenient to fix $\dfrac{d\eta}{ds} = 1$ and then solve the above equation for $\dfrac{d\bm{u}}{ds}$,

$$
\bm{J}(\bm{u}(s), \eta(s)) \dfrac{d\bm{u}}{ds} = - \bm{F}_\eta(\bm{u}(s), \eta(s))
$$


Then, we normalize the tangent vector and choose its orientation such that it matches the orientation of the previous tangent, i.e. such that $\bm{\tau} \cdot \bm{\tau}_0 > 0$. In the first step where we don't have a previously known tangent, we can choose the orientation such that its last element is positive to move forward (in $\eta$) or negative to move backward.

To summarize, we need to solve the following extended system,

$$
\tilde{\bm{F}}(\bm{y}) = 
\begin{pmatrix}
\bm{F}(\bm{y}) \\
(\bm{y} - \bm{y}_0) \cdot \bm{\tau}_0 - \Delta s
\end{pmatrix}
= 0
$$

Its Jacobian is,
$$
\tilde{\bm{J}}(\bm{y}) =
\begin{pmatrix}
\bm{J} & \bm{F}_\eta \\
\dfrac{d\bm{u}}{ds} & \dfrac{d\eta}{ds}
\end{pmatrix}
$$

Now let's see the case of a PDE: the cubic-quintic SHE

$$
\partial_t u = \varepsilon u + u ^ 3 - u ^ 5 - \nu \partial_{xx} u - \partial_{xxxx}u
$$